In [1]:
import pyreadr
import numpy as onp
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import jax.numpy as jnp
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
import time

import sys
import os
# sys.path.append("../../../models") # lab server
sys.path.append("../../../contrastive-regression/models/") # lab server
# from linear_cr import LinearCR
from linear_cr_new import LinearContrastiveRegression

In [2]:
def return_indices_of_a(a, b):
  b_set = set(b)
  return [i for i, v in enumerate(a) if v in b_set]

In [3]:
ct = 'L2or3'
df = pyreadr.read_r('../pb/'+ ct + '.rds')[None]
hvg = pyreadr.read_r('../hvg1k/'+ ct +'.rds')[None]
input1 = df.index.values.tolist()
input2 = hvg[None].to_list()
match_id = return_indices_of_a(input1,input2)
# print(len(match_id))
df = df.iloc[match_id]
print(df.shape)

(1000, 41)


In [4]:
score_type = 'scoreA'
output_dir = './' + score_type + '/'
if not os.path.exists(output_dir):
   os.makedirs(output_dir)

score = pyreadr.read_r('../'+ score_type + '_pfc.rds')[None]
print(score.shape)

(20, 5)


In [5]:
X = df[score['sample'][score['diagnosis']=='ASD'].to_list()].to_numpy().T
print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(10, 1000)


In [6]:
# A = df[score['sample'][score['diagnosis']=='ASD'].to_list()]
# print(list(A) == score['sample'][score['diagnosis']=='ASD'].to_list())
Y = df[score['sample'][score['diagnosis']=='Control'].to_list()].to_numpy().T
print(type(Y))
print(Y.shape)

<class 'numpy.ndarray'>
(10, 1000)


In [7]:
R = score['zscore'][score['diagnosis']=='ASD']
R = onp.atleast_2d(R).T
#print(R)

In [8]:
X_mean, Y_mean, R_mean = onp.mean(X, axis=0), onp.mean(Y, axis=0), onp.mean(R, axis=0)
X = (X - X_mean) / onp.std(X, axis=0)
Y = (Y - Y_mean) / onp.std(Y, axis=0)
R = (R - R_mean) / onp.std(R, axis=0)

In [9]:
# Fit PCA model
d = 8 # dimension reduction

linreg = LinearRegression()
start = time.time()
pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X)
linreg.fit(X_reduced, R)
end = time.time()
preds_linreg = linreg.predict(X_reduced)


In [10]:
# runtime of PCA
runtime = end - start
runtime

0.0034482479095458984

In [11]:
_, _, r2_linreg, _, _ = linregress(R.squeeze(), preds_linreg.squeeze())
print(r2_linreg)

0.999058608278911


In [12]:
components = pca.components_
components.shape

(8, 1000)

In [13]:
# pc1 = components[0]
pcs = ["PC%d" % i for i in range(1,9)]
loading_pc = pd.DataFrame(components, columns = df.index.to_list(), index = pcs,dtype = "float")

In [14]:
pyreadr.write_rds(output_dir + 'PCA.rds', loading_pc, compress="gzip")